# Семинар 2: Библиотека Numpy
### Практикум кафедры ММП ВМК МГУ

О чём можно узнать из этого ноутбука:

* операции при работе с массивами
* многомерные массивы
* изменение размеров массивов
* broadcasting
* продвинутая индексация
* view и копирование
* свёртка
* разные прикладные задачи

## Введение в Python

Чтобы подробнее ознакомиться с языком Python, просмотрите ноутбук **intro_to_python**. В этом ноутбуке мы напомним основные понятия, которые пригодятся на этом занятии.


<font color='brown'>**Задача 1.** Задайте матрицу `list_matrix` размером 500 на 10, в которой ij-ый элемент будет равен 10 * i + j. Не используйте библиотеку numpy! </font>

In [1]:
list_matrix = [[10 * i + j for j in range(10)] for i in range(500)]

Проверьте себя:

In [2]:
assert len(list_matrix) == 500
assert len(list_matrix[0]) == 10
assert list_matrix[10][3] == 103

In [3]:
# А с такими операциями нужно будет очень внимательным! Почему?
b = [[1, 2, 3]] * 10
b

[[1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2, 3]]

In [4]:
# Ответ здесь.
b[0][0] = 10
b

[[10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3],
 [10, 2, 3]]

## Проблемы списков при матричных операциях

Одна из проблем списков — отсутствие поэлементных и матричных операций.

Получим новую матрицу, равную `list_matrix`, умноженной на 2. Измерим время выполнения с помощью магической команды %%timeit:

In [13]:
%%timeit

new_list_matrix = []

for x_vector in list_matrix:
    new_inner = []
    for xy_element in x_vector:
        new_inner.append(xy_element * 2)
    new_list_matrix.append(new_inner)

469 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


А теперь попробуем воспользоваться массивами из библиотеки numpy для аналогичных операций:

In [1]:
# from numpy import * -- НЕ ДЕЛАЙТЕ ТАК
import numpy as np

Текущая версия:

In [15]:
np.__version__

'1.18.5'

С помощью np.zeros зададим нулевой массив нужного размера. Заполним так же, как и list_matrix.

In [16]:
array_matrix = np.zeros((500, 10))

for i in range(500):
    for j in range(10):
        array_matrix[i][j] = j + 10*i
array_matrix[:5]

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.]])

Скоро мы научимся легко использовать конструкции типа:

In [17]:
np.arange(10) + np.arange(0, 41, 10)[:, np.newaxis]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

Все операции с numpy.ndarray по умолчанию поэлементные. Таким образом, умножение на два это всего лишь:

In [18]:
new_array_matrix = 2 * array_matrix

Измерим время выполнения:

In [19]:
%%timeit

new_array_matrix = 2 * array_matrix

2.22 µs ± 77.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Быстрее больше чем в 242 раза...

## Базовые операции

### Создание массивов

Ещё раз обсудим создание массивов. Создание нулевого вектора:

In [429]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

**Внимание**. Если размерность массива > 1, размер подаётся в функцию инициализации через кортеж, а не отдельными аргументами!

Для одномерного массива тоже можно писать кортеж:

In [436]:
np.zeros((5, ))

array([0., 0., 0., 0., 0.])

Так правильно:

In [437]:
np.zeros((5, 7))

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

Так неправильно:

In [438]:
np.zeros(5, 7)

TypeError: data type not understood

Массив из единиц по умолчанию считается вектором-строкой!

In [439]:
np.ones(3).shape

(3,)

Вектор-строка в явном виде:

In [440]:
np.ones((1, 3)).shape

(1, 3)

Вектор-столбец нужно конструировать специальной образом:

In [442]:
np.ones((3, 1)).shape

(3, 1)

Создание массива последовательных чисел:

In [444]:
# похоже на range, но все числа хранятся в памяти
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Создание массива из Python-объекта:

In [85]:
x = np.array([[1, 2, 3], [4, 5, 6]])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [455]:
np.array(list_matrix)

array([[   0,    1,    2, ...,    7,    8,    9],
       [  10,   11,   12, ...,   17,   18,   19],
       [  20,   21,   22, ...,   27,   28,   29],
       ...,
       [4970, 4971, 4972, ..., 4977, 4978, 4979],
       [4980, 4981, 4982, ..., 4987, 4988, 4989],
       [4990, 4991, 4992, ..., 4997, 4998, 4999]])

In [458]:
np.array(range(10))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [460]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Вывести размер полученного массива можно с помощью метода shape:

In [461]:
x.shape

(10,)

<font color='brown'>**Задача 2.** Задайте массив `result` размером 50 на 30, состоящий из троек.</font>

In [467]:
result = np.full((50, 30), 3)

In [79]:
a = np.array([
    [1,2,3],
    [4,5,6]
])
a[:, :, :, np.newaxis]

IndexError: too many indices for array

Проверьте себя:

In [468]:
assert result.shape == (50, 30)
assert np.all(result == 3)

<font color='brown'>**Задача 3.** Какая размерность будет у массива, полученного с помощью команды `np.array([[1], [2]])`?</font>

In [471]:
np.array([[1], [2]]).shape

(2, 1)

In [91]:
# your code here

Можно создать массив из 0, 1 или любого числа, который по размеру и по типу равен какому-то другому:

In [481]:
template = np.array([[4., 5, 6], [1, 2, 3], [7, 8, 9], [10, 11, 12]])
template.shape

(4, 3)

In [482]:
np.zeros_like(template)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [483]:
np.ones_like(template)

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [484]:
np.full_like(template, 42)

array([[42., 42., 42.],
       [42., 42., 42.],
       [42., 42., 42.],
       [42., 42., 42.]])

### Поэлементные операции

In [491]:
x = np.arange(10)
y = np.arange(5, 15)
x, y

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]))

Поэлементные арифметическии операции:

In [487]:
x + y

array([ 5,  7,  9, 11, 13, 15, 17, 19, 21, 23])

In [489]:
x * y

array([  0,   6,  14,  24,  36,  50,  66,  84, 104, 126])

In [492]:
np.sin(x)

array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])

In [496]:
y / x

<ipython-input-496-68e2f9788128>:1: RuntimeWarning: divide by zero encountered in true_divide
  y / x


array([       inf, 6.        , 3.5       , 2.66666667, 2.25      ,
       2.        , 1.83333333, 1.71428571, 1.625     , 1.55555556])

Поэлементые операции сравнения:

In [500]:
x > 5

array([False, False, False, False, False, False,  True,  True,  True,
        True])

### Агрегирующие операции

Агрегация булевого массива
* all — все элементы удовлетворяют требованию
* any — хотя бы один элемент удовлетворяет требованию

In [505]:
x > 5

array([False, False, False, False, False, False,  True,  True,  True,
        True])

In [506]:
np.all(x > 5)

False

In [508]:
np.any(x > 5)

True

Нахождение максимума в массиве:

In [510]:
y

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [511]:
np.max(y)

14

In [517]:
y = np.array([1, 2, 3, 4, 4, 3, 2, 1])
y

array([1, 2, 3, 4, 4, 3, 2, 1])

In [529]:
y.argmax()

3

In [530]:
y.max()

4

Нахождение позиции максимума:

In [531]:
np.argmax(y)

3

Суммирование всех элементов:

In [532]:
np.sum(x)

45

У многих функций в numpy есть "аналогичный" по принципу работы метод: 

In [533]:
x.sum()

45

In [534]:
y.argmax()

3

К массивам можно применять и втроенные в Питон функции, например, sum, однако так делать не стоит, потому что такие операции требуют конвертации к Numpy, и поэтому работают дольше. С другой стороны, встроенные функции (sum) вместе с родными структурами языка (list) будут работать быстрее, чем функции Numpy (np.sum). Именно поэтому лучше не импортировать все функции библиотеки через звёздочку (from numpy import *\), так как это действие перекроет собой встроенные функции Питона.

In [536]:
x = np.arange(10000)

In [117]:
%timeit np.sum(x)

6.41 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [118]:
%timeit sum(x)

1.18 ms ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [538]:
y = range(10000)

In [120]:
%timeit np.sum(y)

757 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [121]:
%timeit sum(y)

147 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Типы и преобразование типов

Атрибут .dtype хранит информацию о типе массива:

In [555]:
x = np.arange(15, dtype=int)
x.dtype

dtype('int32')

In [551]:
y = np.array([1.5, 2.5])
y.dtype

dtype('float64')

In [552]:
np.int32

numpy.int32

In [553]:
int

int

Преобразование типов осуществляется с помощью метода .astype.

Операция изменения типа создаёт копию массива, что требует удвоения памяти и для больших размеров массивов может быть недоступно. 

In [556]:
x = x.astype(np.float64)
x.dtype

dtype('float64')

Можно сразу указывать желаемый тип при создании массивов при помощи параметра dtype:

In [558]:
np.ones((2, 3), dtype=int)

array([[1, 1, 1],
       [1, 1, 1]])

In [559]:
np.ones((2, 3), dtype=str)

array([['1', '1', '1'],
       ['1', '1', '1']], dtype='<U1')

Подробнее о параметрах и действии функции можно посмотреть при помощи ?:

## Размеры массивов

### Изменение размера массивов

NumPy массивы всегда лежат в памяти последовательно, одним куском. Поэтому у массива можно изменять размер практически бесплатно.

In [566]:
x = np.array([[1, 2, 3], [4, 5, 6]])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [565]:
x.reshape?

In [564]:
x.shape

(2, 3)

"Вытянуть" массив в вектор можно с помощью метода .ravel:

In [569]:
x = x.ravel(order="F")
x

array([1, 4, 2, 5, 3, 6])

In [570]:
x.shape

(6,)

Изменить размер массива можно с помощью метда .reshape:

In [571]:
new_x = x.reshape(3, 2) # или x.reshape((3, 2))
new_x

array([[1, 4],
       [2, 5],
       [3, 6]])

In [572]:
new_x.shape

(3, 2)

In [573]:
x.shape

(6,)

Можно "не заполнять" одну размерность итогового массива, она заполнится автоматически:

In [575]:
x.reshape(-1, 6)

array([[1, 4, 2, 5, 3, 6]])

**Это интересно.** Есть два распространённых способа упорядочить многомерный массив:
* C-order, **по умолчанию в NumPy**. Быстрее всего изменяется последний индекс. Для матриц — сначала идём по столбцам.
* Fortran-order (F-order). Быстрее всего изменяется первый индекс. Для матриц — сначала идём по столбцом.

Порядок можно задать с помощью параметра `order` в методе reshape.

In [581]:
x = np.array([[1, 2, 3], [4, 5, 6]])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [582]:
x.T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [583]:
x.reshape(3, 2)

array([[1, 2],
       [3, 4],
       [5, 6]])

### Фиктивная размерность

Два способа добавить фиктивную размерность в массив:

In [584]:
x = np.arange(6)
x

array([0, 1, 2, 3, 4, 5])

In [585]:
x[:, np.newaxis] # Эквивалентно x[:, None]

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])

In [586]:
x.reshape(-1, 1)

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])

In [587]:
x[np.newaxis, :].shape

(1, 6)

In [588]:
x[np.newaxis, :, np.newaxis].shape

(1, 6, 1)

Зачем нужно добавлять фиктивную размерность? Например, чтобы проводить различные операции с массивами неравного размера.

### Повторение массивов

Чтобы построить двумерный массив, состоящий из повторений одномерных, можно использовать np.tile:

In [214]:
a = np.array([2, 3, 4])
np.tile(a[np.newaxis, :], (2, 1))

array([[2, 3, 4],
       [2, 3, 4]])

In [215]:
np.tile(a, (2, 1))

array([[2, 3, 4],
       [2, 3, 4]])

In [216]:
np.tile(a[:, np.newaxis], 2)

array([[2, 2],
       [3, 3],
       [4, 4]])

### Broadcasting

Подробное описание: http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

Пусть нам дана матрица $X$ размером $10 \times 10$ и вектор $y$ длины 10. Пусть мы хотим прибавить вектор к каждой строке матрицы. 

In [589]:
x = np.arange(100).reshape(10, 10)
y = np.arange(10)
x, y

(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

Наивный способ решения проблемы будет работать правильно!

In [590]:
x + y

array([[  0,   2,   4,   6,   8,  10,  12,  14,  16,  18],
       [ 10,  12,  14,  16,  18,  20,  22,  24,  26,  28],
       [ 20,  22,  24,  26,  28,  30,  32,  34,  36,  38],
       [ 30,  32,  34,  36,  38,  40,  42,  44,  46,  48],
       [ 40,  42,  44,  46,  48,  50,  52,  54,  56,  58],
       [ 50,  52,  54,  56,  58,  60,  62,  64,  66,  68],
       [ 60,  62,  64,  66,  68,  70,  72,  74,  76,  78],
       [ 70,  72,  74,  76,  78,  80,  82,  84,  86,  88],
       [ 80,  82,  84,  86,  88,  90,  92,  94,  96,  98],
       [ 90,  92,  94,  96,  98, 100, 102, 104, 106, 108]])

In [591]:
x + y[np.newaxis, :]

array([[  0,   2,   4,   6,   8,  10,  12,  14,  16,  18],
       [ 10,  12,  14,  16,  18,  20,  22,  24,  26,  28],
       [ 20,  22,  24,  26,  28,  30,  32,  34,  36,  38],
       [ 30,  32,  34,  36,  38,  40,  42,  44,  46,  48],
       [ 40,  42,  44,  46,  48,  50,  52,  54,  56,  58],
       [ 50,  52,  54,  56,  58,  60,  62,  64,  66,  68],
       [ 60,  62,  64,  66,  68,  70,  72,  74,  76,  78],
       [ 70,  72,  74,  76,  78,  80,  82,  84,  86,  88],
       [ 80,  82,  84,  86,  88,  90,  92,  94,  96,  98],
       [ 90,  92,  94,  96,  98, 100, 102, 104, 106, 108]])

In [233]:
x + y[:, np.newaxis]

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
       [ 11,  12,  13,  14,  15,  16,  17,  18,  19,  20],
       [ 22,  23,  24,  25,  26,  27,  28,  29,  30,  31],
       [ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42],
       [ 44,  45,  46,  47,  48,  49,  50,  51,  52,  53],
       [ 55,  56,  57,  58,  59,  60,  61,  62,  63,  64],
       [ 66,  67,  68,  69,  70,  71,  72,  73,  74,  75],
       [ 77,  78,  79,  80,  81,  82,  83,  84,  85,  86],
       [ 88,  89,  90,  91,  92,  93,  94,  95,  96,  97],
       [ 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]])

**Почему?** Правила broadcasting (приведение размеров).

1. Если два массива имеют размерности (a_1, a_2 .. a_n) и (b_1, b_2 .. b_n) соответственно, то между ними можно проводить поэлементные операции, если для каждого i выполнено одно из трёх условий:
    * a_i = b_i
    * a_i = 1
    * b_i = 1
    
2. Если поэлементная операция выполняется между массивами разного размера, то к массиву меньшего размера добавляются ведущие фиктивные размерности слева.

**Всегда проверяйте операции с броадкастингом! Легко можно наткнуться на неприятности**

<font color='brown'>**Задача 4.** Какие из этих команд будут выполняться с ошибкой?</font>

1. `np.ones((2, 3)) + np.ones(3)`

2. `np.ones(2) + np.ones((2, 3))`

3. `np.zeros((4, 3)) + np.ones((4, 1))`

4. `np.zeros((3, 4)) + np.ones((4, 3))`

5. `np.zeros((1, 3, 5)) + np.zeros((1, 3))`

6. `np.zeros((5, 3, 1)) + np.zeros((1, 5))`

In [592]:
np.ones(2) + np.ones((2, 3))

array([[2., 2., 2.],
       [2., 2., 2.]])

In [596]:
np.ones((2, 1)) + np.ones((2, 3))

array([[2., 2., 2.],
       [2., 2., 2.]])

In [597]:
np.ones(2)[:, np.newaxis].shape

(2, 1)

In [ ]:
np.ones(2)[:, np.newaxis] + np.ones((2, 3))

In [598]:
np.zeros((4, 3)) + np.ones((4, 1))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [601]:
np.zeros((3, 4)) + np.ones((4, 3))``x.T

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [604]:
np.zeros((1, 3, 5)) + np.zeros((1, 3, 1))

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

<font color='brown'>**Задача 5.** Пусть нам дана матрица $X$ размером $10 \times 10$ и вектор-столбец $y$ длины 10. Получите матрицу result, полученную прибавлением к каждому столбцу $X$ вектора $y$ (без использования циклов). </font>

In [238]:
x = np.arange(100).reshape(10, 10)
y = np.arange(10)

In [609]:
result = x + y[:, np.newaxis]
result

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
       [ 11,  12,  13,  14,  15,  16,  17,  18,  19,  20],
       [ 22,  23,  24,  25,  26,  27,  28,  29,  30,  31],
       [ 33,  34,  35,  36,  37,  38,  39,  40,  41,  42],
       [ 44,  45,  46,  47,  48,  49,  50,  51,  52,  53],
       [ 55,  56,  57,  58,  59,  60,  61,  62,  63,  64],
       [ 66,  67,  68,  69,  70,  71,  72,  73,  74,  75],
       [ 77,  78,  79,  80,  81,  82,  83,  84,  85,  86],
       [ 88,  89,  90,  91,  92,  93,  94,  95,  96,  97],
       [ 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]])

Проверьте себя:

In [610]:
assert result[0][0] == 0
for i in range(1, 10):
    assert result[i][i - 1] == 10 * i + (i - 1) * 2 + 1

## Матричные операции

In [611]:
x = np.array([[1, 2], [2, 1], [2, 3]])
x

array([[1, 2],
       [2, 1],
       [2, 3]])

Транспонирование:

In [612]:
x.T

array([[1, 2, 2],
       [2, 1, 3]])

Матричное умножение:

In [613]:
y = np.array([[0, 1, 0], [1, 0, 1]])

In [614]:
res = np.dot(x, y)
res

array([[2, 1, 2],
       [1, 2, 1],
       [3, 2, 3]])

In [615]:
res = x @ y
res

array([[2, 1, 2],
       [1, 2, 1],
       [3, 2, 3]])

### Операции с размерностями

Некоторые из операций можно принять вдоль некоторых размерностей.

Максимум в каждом столбцe (первая размерность, axis=0):

In [619]:
res = np.arange(12).reshape((4, -1))

In [620]:
res

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [621]:
res.shape

(4, 3)

In [622]:
res.max()

11

In [630]:
np.max(res, axis=0)

array([ 9, 10, 11])

In [631]:
res.sum()

66

In [637]:
res.sum(axis=0)

array([18, 22, 26])

In [26]:
x = np.array([[ 0,  1,  2],
              [ 3,  4,  5],
              [ 6,  7,  8],
              [ 9, 10, 11]])

rows = np.array([[0, 0],
                 [3, 3]], dtype=np.intp)

columns = np.array([[0, 2],
                    [0, 2]], dtype=np.intp)

x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [27]:
x[rows, columns]

array([[ 0,  2],
       [ 9, 11]])

In [28]:
x[rows, columns] = 42

In [32]:
x

array([[42,  1, 42],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [42, 10, 42]])

Как легко запомнить, как работает axis:
* после применения функции получится массив, в котором будет отсутствовать размерность указанная в axis, но остальные размерности будут иметь такие же значения

<font color='brown'>**Задача 6.** Дан вектор $x$ и квадратная матрица $A$. Вычислить вектор значений $y_j = argmin_i (x_i + A_{ij}) \quad \forall j$. </font>

In [638]:
x = np.array([5, 2, 3, 1])
A = np.array([
    [1, 2, 3],
    [2, 2, 4],
    [5, 6, 1],
    [2, 4, 5]
])

In [647]:
y = (x[:, np.newaxis] + A).argmin(axis=0)
y

array([3, 1, 2], dtype=int64)

In [648]:
# your code here

Проверьте себя:

In [649]:
assert y.tolist() == [3, 1, 2]

### Конкатенация матриц

In [650]:
x = np.array([[1, 2], [5, 6]])
y = np.array([[3, 4], [2, 3]])

In [651]:
x

array([[1, 2],
       [5, 6]])

In [652]:
y

array([[3, 4],
       [2, 3]])

Конкатенация матриц по горизонтали:

In [653]:
np.hstack((x, y))

array([[1, 2, 3, 4],
       [5, 6, 2, 3]])

Конкатенация матриц по вертикали:

In [654]:
np.vstack((x, y))

array([[1, 2],
       [5, 6],
       [3, 4],
       [2, 3]])

In [656]:
np.concatenate((x, x, x, x))

array([[1, 2],
       [5, 6],
       [1, 2],
       [5, 6],
       [1, 2],
       [5, 6],
       [1, 2],
       [5, 6]])

Регулируемая конкатенация по какой-то оси:

In [657]:
np.concatenate((x, y), axis=0)

array([[1, 2],
       [5, 6],
       [3, 4],
       [2, 3]])

### Логические маски

Использование масок:

In [658]:
x = np.array([1, 0, 2, 1])
x

array([1, 0, 2, 1])

In [659]:
mask = (x == 1)
mask

array([ True, False, False,  True])

С помощью булевых масок можно выбирать соответствующие элементы массива и даже изменять их:

In [660]:
x[mask]

array([1, 1])

In [662]:
x[mask] = -1
x

array([-1,  0,  2, -1])

In [663]:
x[x == -1] = 42

In [664]:
x

array([42,  0,  2, 42])

С булевыми массивами можно делать побитовые операции с помощью операндов &, |.

С помощью оператора where, можно находить индексы элементов, заданные маской:

In [665]:
y = np.arange(12).reshape(4, -1)

In [666]:
y[y > 4]

array([ 5,  6,  7,  8,  9, 10, 11])

In [674]:
y > 4

array([[False, False, False],
       [False, False,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [676]:
np.where(y > 4)

(array([1, 2, 2, 2, 3, 3, 3], dtype=int64),
 array([2, 0, 1, 2, 0, 1, 2], dtype=int64))

np.where возвращается tuple. Подробнее смотри документацию.

<font color='brown'> **Задача 7.** Даны два вектора одинаковой длины: a и b. Оставить в этих векторах только те элементы, которые соответствуют позициям ненулевых элементов в обоих векторах. </font>

In [680]:
a = np.array([0, 1, 0, 2, 3])
b = np.array([1, 5, 2, 0, 6])

In [684]:
m = (a != 0) & (b != 0)
m

array([False,  True, False, False,  True])

In [688]:
a = a[m]

In [689]:
b = b[m]

In [277]:
# your code here

Проверьте себя:

In [690]:
assert a.tolist() == [1, 3]
assert b.tolist() == [5, 6]

## Индексация

Подробное описание: https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html

In [696]:
x = np.arange(20).reshape(4, 5)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

Простая индексация:

In [697]:
x[1], x[1, :]

(array([5, 6, 7, 8, 9]), array([5, 6, 7, 8, 9]))

Для массивов можно делать slicing, как и для списков. По каждой размерности может быть свой slice.

x[0] эквивалентно x[0, :], то есть недостающие индексы заменяются на :

In [705]:
x[0:3, ::2] = 42

In [706]:
x

array([[42,  1, 42,  3, 42],
       [42,  6, 42,  8, 42],
       [42, 11, 42, 13, 42],
       [15, 16, 17, 18, 19]])

**Это интересно.** Важное отличие от питоновских списков: при slicing возвращается **view**, а не копия! Это позволяет присваивать значения подматрицам.

#### Сложная индексация

По каждой размерности подаются массивы одинаковых размеров, элементы которых соответствуют индексам каждой размерности. Тогда, на выходе будет массив размера этих массивов, элементы которого будут соответствовать элементам исходного массива, взятых в точках, соответствующих поданным массивам.

In [735]:
X = np.array([[1, 2, 3], [4, 5, 6]])
X

array([[1, 2, 3],
       [4, 5, 6]])

In [730]:
np.where(X > 4)

(array([1, 1], dtype=int64), array([1, 2], dtype=int64))

In [732]:
X[np.where(X > 4)[0], np.where(X > 4)[1]] = 42

In [720]:
X[[0, 1], [1, 2]]

array([2, 6])

In [721]:
X[[0, 0], [1, 0]]

array([2, 1])

**Сокращенная индексация**

Если у вас есть многомерный массив, а вам, например, надо взять определенные индексы только с последней размерности, то можно написать так:

In [739]:
x = np.arange(80).reshape((2, 2, 4, 5))

In [740]:
x[..., 0]

array([[[ 0,  5, 10, 15],
        [20, 25, 30, 35]],

       [[40, 45, 50, 55],
        [60, 65, 70, 75]]])

In [288]:
x[..., 0].shape

(2, 2, 4)

## View и копирование

In [777]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [778]:
y = x
y.shape = (2, 5)
x

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [779]:
y

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [780]:
id(x), id(y)

(3035188345472, 3035188345472)

In [781]:
x = x.reshape(5, 2)
x

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [782]:
y

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [783]:
id(x), id(y)

(3035188396720, 3035188345472)

In [784]:
x[:2] = 10

In [785]:
x

array([[10, 10],
       [10, 10],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9]])

In [786]:
y

array([[10, 10, 10, 10,  4],
       [ 5,  6,  7,  8,  9]])

View ссылается на те же данные, но позволяет задать другие размерности массива.

In [763]:
x = np.arange(10)
v = x.view()
v.shape = (2, 5)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [764]:
v

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [765]:
v[0, 0] = 100
x

array([100,   1,   2,   3,   4,   5,   6,   7,   8,   9])

Если нужно получить копию массива, чтобы не портить переданные данные, пользуйтесь функцией copy()

In [766]:
x = np.arange(10)
y = x.copy()
y[:] = 0
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Операция `view` может быть очень полезна в создании комплексных матриц. 

In [767]:
x = np.random.randn(4, 2)
x

array([[-0.3426654 , -0.38973552],
       [ 0.58042895,  0.39798936],
       [-1.413286  ,  0.90862114],
       [ 0.08034333,  0.25097593]])

In [308]:
x.view(complex)

array([[ 2.07865187-0.50566077j],
       [-0.55637846-2.69374696j],
       [-0.83731903-0.42597834j],
       [ 0.2719118 -0.16069585j]])

In [309]:
x.view(complex).shape

(4, 1)

## Разные задачи

<font color='brown'> **Задача 8.** Дана матрица размерности $N \times K$, $N$ - число объектов, $K$ - число признаков.
Подсчитать выборочное среднее и ковариационную матрицу без использования
функции cov.

$$ E[X]={\frac {1}{n}}\sum \limits _{i=1}^{n}x_{i} $$

$$ {\mathrm  {cov}}(X_{{(n)}},Y_{{(n)}})={\frac  1n}\sum _{{t=1}}^{n}\left(X_{t}-\overline {X}\right)\left(Y_{t}-\overline {Y}\right)$$

 </font>

In [127]:
X = np.arange(32).reshape(8, 4)
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]])

In [128]:
mean = X.mean(axis=0)
mean

array([14., 15., 16., 17.])

In [129]:
# n = X.shape[0]
# верхняя формула раскрывается таким образом (линал)
# из нампая здесь только операция матричного умножения @

cov = (X - mean).T @ (X - mean) / X.shape[0]
cov

array([[84., 84., 84., 84.],
       [84., 84., 84., 84.],
       [84., 84., 84., 84.],
       [84., 84., 84., 84.]])

Проверьте себя:

In [130]:
assert mean.tolist() == [14, 15, 16, 17]
assert np.all(cov == 84)

<font color='brown'> **Задача 9.** Замените все максимальные элементы матрицы $A$ на 0. </font>

In [86]:
A = np.array([
    [1, 6, 3],
    [6, 2, 1],
    [4, 5, 6],
    [1, 2, 1],
])

In [88]:
index = (A == A.max()) # нашли индексы максимальных (бинарную маску)
index

array([[False, False, False],
       [False, False, False],
       [False,  True, False],
       [False, False, False]])

In [87]:
## your code
A[index] = 0 # зануляем элементы по этим индексам (бинарной маске)
A

array([[1, 0, 3],
       [0, 2, 1],
       [4, 5, 0],
       [1, 2, 1]])

In [89]:
new_A = np.array([
    [1, 0, 3],
    [0, 2, 1],
    [4, 5, 0],
    [1, 2, 1],
])
assert np.all(A == new_A)

<font color='brown'> **Задача 10.** Вычислить площадь круга методом Монте-Карло. </font> </font>

In [90]:
N = 10000
X = np.random.uniform(0, 1, size=(N, 2))
X

array([[0.17940125, 0.47372453],
       [0.82575589, 0.98343335],
       [0.66996628, 0.38873857],
       ...,
       [0.63458342, 0.88479986],
       [0.81508141, 0.2069276 ],
       [0.40049934, 0.68086431]])

In [91]:
M = np.array([0.5, 0.5]) # центр квадрата и вписанной окружности
M

array([0.5, 0.5])

In [93]:
delta = (X - M)**2 # считаем покоординатные квадраты разностей объектов с центром M
delta

array([[0.10278356, 0.0006904 ],
       [0.1061169 , 0.2337078 ],
       [0.02888854, 0.01237911],
       ...,
       [0.0181127 , 0.14807093],
       [0.09927629, 0.08589143],
       [0.00990038, 0.0327119 ]])

In [94]:
dist2center = delta.sum(axis=1)**(0.5) # суммируем квадраты покоординатных разностей и берем корень
dist2center # получаем евклидово расстояние до центра M

array([0.32167368, 0.58294485, 0.20314439, ..., 0.40765626, 0.4303112 ,
       0.20642742])

In [97]:
r = 0.5 # радиус вписанной окружности
lower_than_r = (dist2center < r).mean() # считаем отношение числа точек попавших в круг к количеству всех точек
# что является отношением площади круга к площади квадрата
lower_than_r # поскольку площадь квадрата равна 1, площадь круга lower_than_r * 1 = lower_than_r

0.7844

Проверьте себя:

In [99]:
np.pi / 4

0.7853981633974483

<font color='brown'> **Задача (со звёздочкой) 11.** Даны матрицы $A$ размера $(n \times d)$ и $B$ размера $(m \times d)$. Найти в A все строки, содержащиеся в B, не используя циклы. Оцените сложность полученного алгоритма.  (сначала смотрите задачу 12)</font>

In [118]:
A = np.array([
    [1, 2, 3],
    [1, 2, 1],
    [4, 5, 6],
    [1, 2, 1],
    [4, 2, 1]
])

B = np.array([
    [4, 5, 6],
    [4, 5, 6],
    [1, 2, 3],
    [0, 1, 2]
])

In [122]:
# A и B - вектора объектов
# теперь после 12-ой - задачи мы умеем находить расстояния между объектами двух векторов
# в данном случае воспользуемся расстоянием манхэттона (погуглите что это)
# используем его чтобы не переходить к вещественным расстояниям

MAIN_MATRIX = B - A[:,np.newaxis,:] # поначалу все как в евклидовом
MAIN_MATRIX = abs(MAIN_MATRIX)   # берем по модулю
dist = MAIN_MATRIX.sum(axis=2) # получаем матрицу расстояний. по строкам - объекты вектора B
# по столбцам - объекты матрицы А
dist

array([[ 9,  9,  0,  3],
       [11, 11,  2,  3],
       [ 0,  0,  9, 12],
       [11, 11,  2,  3],
       [ 8,  8,  5,  6]])

In [123]:
distance2nearest = dist.min(axis=1) # для каждого объекта B - расстояния до ближайшего 
distance2nearest                    # объекта из вектора А.

array([0, 2, 0, 2, 5])

In [125]:
# если между объектами расстояние манхэттона 0, то они совпадают
# проверяем на совпадение объектов. 
indexes = np.where(distance2nearest == 0)[0] #
indexes

array([0, 2], dtype=int64)

Проверьте себя:

In [126]:
assert set(indexes) == {0, 2}

<font color='brown'> **Задача (со звёздочкой) 12.** Даны матрицы $A$ размера $(n \times d)$ и $B$ размера $(m \times d)$. Посчитайте матрицу попарных расстояний D размера $(n \times m)$ такую, что $d_{ij} = \{||a_{i} - b_{j}||\}_{i,j}^{n, m}$,  используя broadcasting. Оцените сложность полученного алгоритма.  
  

In [104]:
m, n, d = 5, 4, 3

A = np.random.randn(n, d)
B = np.random.randn(m, d)
A, B

(array([[ 0.12468989, -1.12560846,  1.71629357],
        [ 0.81065962, -0.00793974,  1.06011379],
        [ 0.84176087,  0.01173753, -0.18029645],
        [ 0.27692493, -0.10477078, -1.69992722]]),
 array([[-2.1787208 ,  0.79253446, -1.5482931 ],
        [-0.29086113, -0.89449614, -0.13029802],
        [-0.2408462 , -0.35012025, -0.18771432],
        [-1.11737167, -0.8284814 ,  0.02500734],
        [-1.15417432, -0.70291541,  0.17876782]]))

## как такое решать???

давайте упростим. тут двумерный тензор (матрица). Проще говоря, вектор векторов
[ [...],  [...],  [...] ]

для начала нужно получить все возможные разности объектов в векторе (в данном случае векторов), а для этого в numpy используется фиктивная размерность (np.newaxis) или broadcasting (два способа говорить об одном и том же). самый простой пример их использования:

a[:, np.newaxis, :)]    + a
[[...],             + [ ..., ..., ...]
 [...],
 [...]]  
 
при этом ... - по типу может быть не только числом, но и любым n - мерным тензором. Для сохранения общности будем называть их фиксированными объектами.
В данной операции мы держим в голове, что имеем вектор объектов. Нам нужно провести операцию вектор-столбец на вектор-строку (элементы вектор-столбца и вектор-строки - объекты). Чтобы провести эту операцию мы должны поставить фиктивную ось сразу после той, вдоль которой находятся наши объекты. В случае объектов - чисел, мы делали так a[:,np.newaxis], так как сами по себе числа одноразмерны (нет индексации вдоль объекта). В этом случае у нас есть индексация внутри объекта, поэтому фиктивная ось должна быть перед ней (по сути, мы не нарушаем никак индексацию объекта, так как наша цель их зафиксировать). поэтому конструкция будет выглядеться так:
a[:,np.newaxis,:]: 0-ая ось - индексируемся вдоль объектов. 1-ая ось - фиктивная (для того чтобы представить обычный вектор размерности (n), как вектор столбец размерности (n,1) ). 2-ая ось - индексация внутри фиксированного объекта.

Точно таким же образом мы могли совершать операции, если бы у объектов было бы больше осей внутри. Если бы объекты были матрицами, а не векторами, то было бы a[:,np.newaxis,:,:], трехмерными тензорами - a[:,np.newaxis,:,:,:] и так далее.

In [112]:
# your code
# вся суть задачи
MAIN_MATRIX = A - B[:,np.newaxis, :] # получаем матрицу объектов (векторов)
MAIN_MATRIX

array([[[ 2.30341069, -1.91814293,  3.26458666],
        [ 2.98938042, -0.80047421,  2.60840688],
        [ 3.02048167, -0.78079693,  1.36799664],
        [ 2.45564573, -0.89730524, -0.15163413]],

       [[ 0.41555102, -0.23111233,  1.84659159],
        [ 1.10152075,  0.8865564 ,  1.19041181],
        [ 1.132622  ,  0.90623367, -0.04999843],
        [ 0.56778606,  0.78972536, -1.5696292 ]],

       [[ 0.36553609, -0.77548821,  1.90400789],
        [ 1.05150582,  0.34218051,  1.24782811],
        [ 1.08260707,  0.36185778,  0.00741787],
        [ 0.51777113,  0.24534947, -1.5122129 ]],

       [[ 1.24206157, -0.29712707,  1.69128622],
        [ 1.92803129,  0.82054166,  1.03510644],
        [ 1.95913255,  0.84021893, -0.2053038 ],
        [ 1.3942966 ,  0.72371062, -1.72493456]],

       [[ 1.27886421, -0.42269305,  1.53752575],
        [ 1.96483394,  0.69497567,  0.88134596],
        [ 1.99593519,  0.71465295, -0.35906428],
        [ 1.43109925,  0.59814464, -1.87869504]]])

In [114]:
# дальше все относительно просто
quadratic_delta = MAIN_MATRIX**2 # берем квадраты поэлементных разностей
delta = quadratic_delta.sum(axis=2) # суммируем квадраты покоординатных разностей вдоль осей объекта
# в данном случае у объекта одна ось, поэтому суммируем только по ней
euclidean_distance = delta**(0.5) # берем корень от суммы и получаем евклидово расстояние
euclidean_distance

array([[4.43198592, 4.04733748, 3.40651848, 2.61884431],
       [1.9068288 , 1.84835345, 1.45141032, 1.84655977],
       [2.08811989, 1.66728131, 1.14150521, 1.61711816],
       [2.11930426, 2.33710047, 2.14156901, 2.3330708 ],
       [2.04405198, 2.26281557, 2.15021238, 2.43625063]])